In [ ]:
import pandas as pd
import re

df = pd.read_csv("../datas/tanner_report.csv")

patterns = {
    # "RFI": r"(?:https?|ftp):\/\/(?!202\.10\.35\.215)([\w.-]+)(?:\/|$)",
    # "LFI": r"(\.\.\/)+|\/(etc|proc|var|home|root|dev|tmp)\/",
    # "XSS": r"(?i)(<script.*?>.*?</script>|<.*?on\w+\s*=.*?>|javascript:|%3Cscript|alert\s*\(|document\.cookie|<\/script>)",
    # "SQLi": r"(?i)(\bselect\b.*?\bfrom\b|\binsert\b.*?\binto\b|\bunion\b.*?\bselect\b|drop\s+table|--|\bOR\b\s+\d+=\d+|sleep\s*\(|benchmark\s*\(|\bupdate\b.*?\bset\b)",
    # "Command Injection": r"(;|\|\||&&|\b(cat|ls|wget|curl|whoami|ping|nc|bash|sh|python|perl)\b)",
    # "PHP Code Injection": r"(?i)(eval\s*\(|assert\s*\(|system\s*\(|passthru\s*\(|base64_decode\s*\(|shell_exec\s*\(|`[^`]*`)",
    # "PHP Object Injection": r"O:\d+:\"[^\"]+\":\d+:\{.*?\}",
    # "CRLF": r"(%0d%0a|\r\n|\n\r|\r|\n)",
    # "XXE": r"(?i)<\?xml.*?\?>|<!DOCTYPE\s+[a-zA-Z]+.*?>",
    # "Template Injection": r"(\{\{.*?\}\}|\{%.+?%\}|__proto__|constructor\s*\(|toString\s*\()"

    "LFI": r"(\.\./)+|/etc/passwd|/proc/self/environ",
    "RFI": r"(https?:)?//[^\s]+(\.php|\.txt)?",
    "SQLi": r"('|\"|;|--|\b(OR|AND|SELECT|UNION|INSERT|UPDATE|DROP|WHERE|HAVING|BENCHMARK)\b)",
    "XSS": r"(<script.*?>.*?</script>|<[^>]+on\w+=|javascript:|data:text/html|&#x?[0-9a-f]+;)",
    "Command Injection": r"(;|\||&&|`|\$\(.*?\)|\b(cat|ls|whoami|nc|curl|wget)\b)",
    "PHP Code Injection": r"<\?(php)?\s+.*?\?>",
    "PHP Object Injection": r"O:\d+:\"[A-Za-z0-9_]+\":\d+:\{.*?\}",
    "CRLF": r"%0d%0a|%0a|%0d|\r\n|\n|\r",
    "XXE": r"(<!DOCTYPE\s+[^>]*\[\s*<!ENTITY\s+)",
    "Template Injection": r"({[{%].*?[}%]})",
}

def detect_attack(path, detection_name=None):
    if detection_name is not None:
        if detection_name.lower() == "sqli":
            return "SQLi"
        elif detection_name.lower() == "xss":
            return "XSS"
    for attack_type, pattern in patterns.items():
        if re.search(pattern, path, re.IGNORECASE):
            return attack_type
    return "Unknown"

d_attack = {
    "Unknown": 0,
    "RFI": 1,
    "LFI": 2,
    "XSS": 3,
    "SQLi": 4,
    "Command Injection": 5,
    "PHP Code Injection": 6,
    "PHP Object Injection": 7,
    "CRLF": 8,
    "XXE": 9,
    "Template Injection": 10
}

In [8]:
df["attack_name"] = df.apply(lambda row: detect_attack(row["path"], row["detection_name"]), axis=1)
df["attack_type"] = df["attack_name"].map(d_attack)

# df["attack_name"] = df["path"].apply(detect_attack)
# df["attack_type"] = df["attack_name"].map(d_attack)

In [9]:
df.to_csv('detected.csv')